## Import

In [ ]:
!pip install autogluon

In [ ]:
import random
import pandas as pd
import numpy as np
import os
from tqdm.auto import tqdm
import librosa

from autogluon.tabular import TabularDataset, TabularPredictor
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

import warnings
warnings.filterwarnings(action='ignore')

from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


## Hyperparameter Setting

In [ ]:
CFG = {
    'SR':16000,
    'N_MFCC':128, # Melspectrogram 벡터를 추출할 개수
    'SEED':812
}

## Fixed Random-Seed

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(CFG['SEED']) # Seed 고정

## Data Pre-Processing

In [ ]:
!unzip -qq "/content/drive/MyDrive/Dacon/월간 데이콘 음성 감정 인식 AI 경진대회/data/open.zip"

In [ ]:
test_df = pd.read_csv("/content/open/test.csv")
train_df = pd.read_csv("/content/open/train.csv")

In [ ]:
test_df['path'] = test_df['path'].str.replace('./test', '/content/open/test')
train_df['path'] = train_df['path'].str.replace('./train', '/content/open/train')

In [ ]:
train_df['path'].head()

0    /content/open/train/TRAIN_0000.wav
1    /content/open/train/TRAIN_0001.wav
2    /content/open/train/TRAIN_0002.wav
3    /content/open/train/TRAIN_0003.wav
4    /content/open/train/TRAIN_0004.wav
Name: path, dtype: object

In [ ]:
test_df['path'].head()

0    /content/open/test/TEST_0000.wav
1    /content/open/test/TEST_0001.wav
2    /content/open/test/TEST_0002.wav
3    /content/open/test/TEST_0003.wav
4    /content/open/test/TEST_0004.wav
Name: path, dtype: object

In [ ]:
def get_mfcc_feature(df):
    features = []
    for path in tqdm(df['path']):
        y, sr = librosa.load(path, sr=CFG['SR'])
        mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=CFG['N_MFCC'])
        features.append({
            'mfcc_mean': np.mean(mfcc, axis=1),
            'mfcc_max': np.max(mfcc, axis=1),
            'mfcc_min': np.min(mfcc, axis=1),
        })

    mfcc_df = pd.DataFrame(features)
    mfcc_mean_df = pd.DataFrame(mfcc_df['mfcc_mean'].tolist(), columns=[f'mfcc_mean_{i}' for i in range(CFG['N_MFCC'])])
    mfcc_max_df = pd.DataFrame(mfcc_df['mfcc_max'].tolist(), columns=[f'mfcc_max_{i}' for i in range(CFG['N_MFCC'])])
    mfcc_min_df = pd.DataFrame(mfcc_df['mfcc_min'].tolist(), columns=[f'mfcc_min_{i}' for i in range(CFG['N_MFCC'])])

    return pd.concat([mfcc_mean_df, mfcc_max_df, mfcc_min_df], axis=1)


def get_feature_mel(df):
    features = []
    for path in tqdm(df['path']):
        data, sr = librosa.load(path, sr=CFG['SR'])
        n_fft = 2048
        win_length = 2048
        hop_length = 1024
        n_mels = 128

        D = np.abs(librosa.stft(data, n_fft=n_fft, win_length = win_length, hop_length=hop_length))
        mel = librosa.feature.melspectrogram(S=D, sr=sr, n_mels=n_mels, hop_length=hop_length, win_length=win_length)

        features.append({
            'mel_mean': mel.mean(axis=1),
            'mel_max': mel.min(axis=1),
            'mel_min': mel.max(axis=1),
        })
    mel_df = pd.DataFrame(features)
    mel_mean_df = pd.DataFrame(mel_df['mel_mean'].tolist(), columns=[f'mel_mean_{i}' for i in range(n_mels)])
    mel_max_df = pd.DataFrame(mel_df['mel_max'].tolist(), columns=[f'mel_max_{i}' for i in range(n_mels)])
    mel_min_df = pd.DataFrame(mel_df['mel_min'].tolist(), columns=[f'mel_min_{i}' for i in range(n_mels)])

    return pd.concat([mel_mean_df, mel_max_df, mel_min_df], axis=1)

In [ ]:
train_mf = get_mfcc_feature(train_df)
test_mf = get_mfcc_feature(test_df)

  0%|          | 0/5001 [00:00<?, ?it/s]

  0%|          | 0/1881 [00:00<?, ?it/s]

In [ ]:
train_mel = get_feature_mel(train_df)
test_mel = get_feature_mel(test_df)

  0%|          | 0/5001 [00:00<?, ?it/s]

  0%|          | 0/1881 [00:00<?, ?it/s]

In [ ]:
train_x = pd.concat([train_mel, train_mf], axis=1)
test_x = pd.concat([test_mel, test_mf], axis=1)

In [ ]:
train_y = train_df['label']

In [ ]:
train_x['label'] = train_df['label']
train_data = TabularDataset(train_x)
test_data = TabularDataset(test_x)

In [ ]:
train_x

,mel_mean_0,mel_mean_1,mel_mean_2,mel_mean_3,mel_mean_4,mel_mean_5,mel_mean_6,mel_mean_7,mel_mean_8,mel_mean_9,...,mfcc_min_119,mfcc_min_120,mfcc_min_121,mfcc_min_122,mfcc_min_123,mfcc_min_124,mfcc_min_125,mfcc_min_126,mfcc_min_127,label
0,0.084566,0.056834,0.068925,0.102323,0.096968,0.086162,0.094872,0.122292,0.132902,0.129273,...,-3.878135,-4.017256,-4.249370,-3.223007,-4.603190,-1.934262,-3.485540,-2.463422,-3.229237,1
1,0.098563,0.052329,0.056681,0.061517,0.081819,0.106868,0.138239,0.107013,0.144104,0.120996,...,-3.810816,-4.056769,-4.705098,-4.370795,-2.665024,-5.128586,-3.855324,-3.354258,-3.015537,2
2,0.083089,0.055145,0.060235,0.078781,0.078231,0.141836,0.142165,0.156169,0.129583,0.136608,...,-4.208198,-4.048994,-3.270410,-3.567141,-5.038787,-3.207233,-5.000182,-4.171638,-3.029602,4
3,0.020401,0.055993,0.088625,0.112177,0.166121,0.092082,0.082991,0.106938,0.118478,0.130794,...,-4.395284,-2.994615,-4.725981,-3.209175,-4.101068,-5.370553,-3.596648,-4.457970,-4.496693,5
4,0.074340,0.047811,0.056292,0.110727,0.126938,0.169690,0.195548,0.141796,0.155485,0.272525,...,-2.485119,-3.151915,-4.735774,-3.539431,-3.760096,-3.584368,-3.163036,-4.498411,-4.766491,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4996,0.021166,0.058369,0.111723,0.110858,0.119897,0.068029,0.117166,0.131592,0.119517,0.130425,...,-2.708395,-4.064009,-4.244309,-3.986596,-3.176047,-3.143502,-4.341455,-2.936268,-3.653713,5
4997,0.094659,0.061243,0.070688,0.099395,0.105742,0.081150,0.080464,0.094489,0.137502,0.163564,...,-3.696562,-3.079942,-6.046649,-5.025594,-4.648947,-2.863223,-3.787347,-3.878422,-3.689405,0
4998,0.120308,0.064221,0.062565,0.062162,0.088277,0.158967,0.121889,0.090846,0.138761,0.148893,...,-2.578660,-4.233519,-4.165043,-3.027788,-5.933723,-2.864382,-4.454406,-3.263405,-4.237404,1
4999,0.074006,0.055329,0.062009,0.098595,0.098970,0.073677,0.073534,0.102842,0.120538,0.151261,...,-4.765526,-5.089233,-4.338481,-4.685366,-4.513369,-5.112648,-3.516397,-3.337690,-4.203058,1


# autogluon

In [ ]:
label = 'label'
eval_metric = 'accuracy'
time_limit = 3600 * 11

In [ ]:
predictor = TabularPredictor(
    label=label, eval_metric=eval_metric, path="/content/drive/MyDrive/Dacon/월간 데이콘 음성 감정 인식 AI 경진대회/model"
).fit(train_data, presets='best_quality', time_limit=time_limit)

Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=20
Beginning AutoGluon training ... Time limit = 39600s
AutoGluon will save models to "/content/drive/MyDrive/Dacon/월간 데이콘 음성 감정 인식 AI 경진대회/model/"
AutoGluon Version:  0.7.0
Python Version:     3.10.11
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Sat Apr 29 09:15:28 UTC 2023
Train Data Rows:    5001
Train Data Columns: 768
Label Column: label
Preprocessing data ...
AutoGluon infers your prediction problem is: 'multiclass' (because dtype of label-column == int, but few unique label-values observed).
	6 unique label values:  [1, 2, 4, 5, 3, 0]
	If 'multiclass' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Train Data Class Count: 6
Using Feature Generators to preprocess the data ...
Fitting Au

FileNotFoundError: ignored

In [ ]:
predictor.leaderboard(silent=True)

## Inference

In [ ]:
model_to_use = predictor.get_model_best()

model_pred = predictor.predict(test_data, model=model_to_use)

In [ ]:
model_to_use = "WeightedEnsemble_L2"
model_pred = predictor.predict(test_data, model=model_to_use)

## Submission

In [ ]:
submission = pd.read_csv('/content/open/sample_submission.csv')
submission['label'] = model_pred
submission.to_csv('/content/drive/MyDrive/Dacon/월간 데이콘 음성 감정 인식 AI 경진대회/result/autogluon_1st_0530.csv', index=False)